*LLM - GPT Model*

This is a study case of LLMs, developing and training a local GPT model using Open/Free content.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [2]:
print(torch.version.cuda)

12.1


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#Definitions / hyperparameters
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layers = 4   #Normaly same number of decoders
dropout = 0.2

cuda


In [4]:
%%time
start_time = time.time()
zeros = torch.zeros(100,100,100,100)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.8f}")

0.07024932
CPU times: total: 203 ms
Wall time: 70.2 ms


In [5]:
chars = ""
with open('../Dataset/Books/The_Wizard_of_OS.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

#print(text[:200])


*Tokenizing*
These blocks of code are tokenizing the book, to extract each char and creating a SET

Using enconde/decode functions, we can save a lot of memory since we don't need to

work with all tokens (only with that tokens that are present on the dataset)

In [6]:
#chars = sorted(set(text))  #Tokenizing - Verify the vocabulary (chars) used on this book 
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [7]:
#Creating decoders and encoders to easily convert data from String to int and int to string

string_to_int = { ch:i for i, ch in enumerate(chars)} #Getting the string
int_to_string = { i:ch for i,ch in enumerate(chars)} #Getting the ints
encode = lambda s: [string_to_int[c] for c in s]  #Encoding from string to int
decode = lambda l: ''.join([int_to_string[i] for i in l])  #Decoding from int to string



In [8]:
#Testing
#The expected result should be return a LIST with the index value for each char
encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [9]:
#Creating a tensor with our encoded text
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [10]:
#Splitting the dataset (80-20)
n  = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets')
print(y)

inputs: 
tensor([[57,  1, 73,  ..., 61,  1, 73],
        [65, 62, 59,  ..., 66, 11,  1],
        [ 0, 76, 68,  ..., 65, 57,  1],
        ...,
        [58,  1, 61,  ..., 68, 76, 54],
        [ 1, 54,  1,  ..., 71,  1, 73],
        [58,  1, 71,  ..., 72,  1, 56]], device='cuda:0')
targets
tensor([[ 1, 73, 61,  ...,  1, 73, 68],
        [62, 59, 68,  ..., 11,  1, 33],
        [76, 68, 74,  ..., 57,  1, 72],
        ...,
        [ 1, 61, 68,  ..., 76, 54, 71],
        [54,  1, 60,  ...,  1, 73, 61],
        [ 1, 71, 58,  ...,  1, 56, 68]], device='cuda:0')


In [12]:
@torch.no_grad()  #Reducing computation
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

- B: Batch
- T: Time
- C: Channel

In [13]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  #key transformation
        q = self.query(x)
        #Compute attention scores
        # Matrix coalculation - query by key (transposed)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #(B, T, hs) @ (B, hs, T) -> B, T, T
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out
        

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [15]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
             nn.Linear(n_embd, 4 * n_embd),
             nn.ReLU(),
             nn.Linear(4 * n_embd, n_embd),
             nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)   #Self Attentiion
        x = self.ln1(x + y)  #Linear Normalization
        y = self.ffwd(x)  #Feed Forward
        x = self.ln2(x + y) #Linear Normalization
        return x

In [17]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        #Blocs
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layers)])  #Creating the decoding blocks
        self.ln_f = nn.LayerNorm(n_embd)   #Final normalization Layer
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  #(T, C)
        x = tok_emb + pos_emb    #(B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indexes in the current context
        for _ in range(max_new_tokens):
            #Getting predictions
            logits, loss = self.forward(index)
            #Focus only on the last time step
            logits = logits[:, -1, :]
            #Apply softmax to get probs
            probs = F.softmax(logits, dim=-1)
            #Sample from distrib
            index_next = torch.multinomial(probs, num_samples=1)
            #Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)  #(B, T+1)
        return index
            

model = GPTLanguageModel(vocab_size)
m = model.to(device)

#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)


In [18]:
#Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    
    xb, yb = get_batch('train')

    #evaluate
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss 4.431, val loss 4.430
step: 100, train loss 2.237, val loss 2.308
step: 200, train loss 1.897, val loss 2.001
step: 300, train loss 1.689, val loss 1.831
step: 400, train loss 1.566, val loss 1.730
step: 500, train loss 1.473, val loss 1.659
step: 600, train loss 1.398, val loss 1.614
step: 700, train loss 1.349, val loss 1.577
step: 800, train loss 1.298, val loss 1.554
step: 900, train loss 1.254, val loss 1.535
step: 1000, train loss 1.221, val loss 1.522
step: 1100, train loss 1.185, val loss 1.509
step: 1200, train loss 1.153, val loss 1.503
step: 1300, train loss 1.125, val loss 1.497
step: 1400, train loss 1.097, val loss 1.489
step: 1500, train loss 1.070, val loss 1.490
step: 1600, train loss 1.048, val loss 1.499
step: 1700, train loss 1.018, val loss 1.496
step: 1800, train loss 0.993, val loss 1.496
step: 1900, train loss 0.971, val loss 1.506
step: 2000, train loss 0.944, val loss 1.508
step: 2100, train loss 0.926, val loss 1.519
step: 2200, train loss

In [19]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2